# TP4_depart
## IFT870/BIN710 Hiver 2021

### Mathieu Lussier (LUSM1202)
### Gabriel Gibeau Sanchez (GIBG2501)

In [1]:
import sklearn
import numpy as np
import scipy as sp
import pandas as pd
%matplotlib notebook
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.neighbors import  KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

from itertools import product
from re import finditer

# for printing variables in a markdown cell
from IPython.display import Markdown as md

In [2]:
TP4_data_file = "RNA_orignal.csv"
TP4_data = pd.read_csv(TP4_data_file,index_col=0)
TP4_data.shape

(395, 2)

In [3]:
TP4_data[0:10]

,sequence,classe
id,,
AF093014.1/662-809,CACUUUAAGUAAUGGAUGUCUUGGUUCUCAUAACGAUGAAGGACGC...,RF00002
M14649.1/2015-2169,AACUUUCAGCGACGGAUGUCUCGGCUCGAACAACGAUGAAGGGCGC...,RF00002
X53361.2/1206-1368,ACCUGUUGUGGUGGAUGUCUUGGCCCAGGUUCUGAGGAAGGACACA...,RF00002
M86760.1/218-371,AACUUUCAGCAAUGGAUCUCUUGGUUCCCGCGUCGAUGAAGAACGU...,RF00002
X00601.1/3997-4154,AAGCAUAAACGGUGAAUACCUCGACUCCUAAAUCGAUGAAGACCGU...,RF00002
U48228.1/7-166,CAAUCUUAACGAUGGAUGUCUUGGUUCCUAUAGCGAUGAAGGCCGC...,RF00002
U58510.1/2022-2198,AACCCUAGUGAUGGAUGUCUAGGCUCCCGUAUCGAUGAAGAACGUA...,RF00002
AL049755.2/30863-31016,AACUUUCAGCAACGGAUCUCUUGGCUCUCGCAUCGAUGAAGAACGC...,RF00002
U13369.1/6624-6776,GACUCUUAGCGGUGGAUCACUCGGCUCGUGCGUCGAUGAAGAACGC...,RF00002


# Q1
## a)
Un examen des données initiales revèle que les entrées dont la classes est absente contiennent en fait la classe dans la
colonne "sequence", concaténée à la valeur de séquence génétique mais séparée par une virgule. Ce puisque les champs du
fichier sont contenus entre des guillements doubles, mais pour ces entrées un guillemet ouvrant au début de la séquence génétique
ne trouve pas son homologue fermant (il devrait être à la fin de la séquence) ce qui fait en sorte que la virgule est vue
comme un simple caractère de ponctuation, au lieu de servir de séparateur pour le format CSV.

Pour remédier à ce problème, nous utilisons les fonctions de la classe $\textit{string}$ et nous sauvegardons le
$\textit{DataFrame}$ corrigé dans un nouveau fichier CSV intitulé "RNA.csv".

In [4]:
# Get indexes for which the entries have no classe
na_indexes = TP4_data['classe'].isna()

data = TP4_data.copy()

# Get missing classes & cleaned sequences
split_vals = TP4_data.loc[na_indexes, 'sequence'].str.split(pat='[,"]', expand=True)
data.loc[na_indexes, 'classe'] = split_vals.iloc[:, 1]
data.loc[na_indexes, 'sequence'] = split_vals.iloc[:, 0]

# Save cleaned data
data.to_csv('RNA.csv')

# We'll free up unused variables to keep our dev space uncluttered
del split_vals
del na_indexes

## b)

Il y a 12 classes dont la répartition est donnée dans la cellule de code suivante:

In [5]:
# Ceckout unique classes values and instances number for each classe
print(f'Il y a {data.classe.unique().shape[0]} classes')
for ix, cls in enumerate(data.classe.value_counts().index):
    print(f'La classe {cls} contient {data.classe.value_counts()[ix]} entrées')

Il y a 12 classes
La classe RF00008 contient 54 entrées
La classe RF00004 contient 51 entrées
La classe RF00017 contient 49 entrées
La classe RF00002 contient 46 entrées
La classe RF00003 contient 42 entrées
La classe RF00023 contient 36 entrées
La classe RF00015 contient 25 entrées
La classe RF00024 contient 24 entrées
La classe RF00011 contient 19 entrées
La classe RF00012 contient 17 entrées
La classe RF00019 contient 16 entrées
La classe RF00025 contient 16 entrées


## c)

Ici nous balançons le nombre de données en se défaussant des instances excédant le nombre minimal

In [6]:
# Fint out the minimum instances number
classes_rep = data.classe.value_counts()
print(f'Le nombre minimum d\'instance d\'une classe est {classes_rep.min()} pour la classe:\n'
      f'{classes_rep[classes_rep == classes_rep.min()].index.values}')

cut_off = classes_rep.min()

# Trim off exceding instances
for cls in classes_rep.index:
    indexes_to_drop = data[data['classe'] == cls].index
    if indexes_to_drop.shape[0] > cut_off:
        data.drop(index=indexes_to_drop[16:], axis=0, inplace=True)

print(f'Après traitement, les classes contiennent maintenant {cut_off} instance chacune:')
for ix, cls in enumerate(data.classe.value_counts().index):
    print(f'La classe {cls} contient {data.classe.value_counts()[ix]} entrées')

del classes_rep
del cut_off

Le nombre minimum d'instance d'une classe est 16 pour la classe:
['RF00019' 'RF00025']
Après traitement, les classes contiennent maintenant 16 instance chacune:
La classe RF00012 contient 16 entrées
La classe RF00025 contient 16 entrées
La classe RF00003 contient 16 entrées
La classe RF00023 contient 16 entrées
La classe RF00015 contient 16 entrées
La classe RF00011 contient 16 entrées
La classe RF00008 contient 16 entrées
La classe RF00019 contient 16 entrées
La classe RF00002 contient 16 entrées
La classe RF00024 contient 16 entrées
La classe RF00004 contient 16 entrées
La classe RF00017 contient 16 entrées


# Q2
## a)
Ici nous ajoutons une colonne à notre $\textit{dataframe}$ pour le nouvel attribut "longueur". Nous allons également encoder
les classes en format numérique pour pouvoir calculer aisément la justesse des prédictions faites par notre modèle de
$\textit{K-Nearest-Neighbors}$.

In [7]:
# add longueur column
data['longueur'] = data['sequence'].str.len().values

# Encode classes labels to numerical values
lencoder = LabelEncoder()
data['encoded_classe'] = lencoder.fit_transform(data['classe'].values)

# Prepare training/test data, train and predict
X_train, X_test, y_train, y_test = train_test_split(data['longueur'].values,
                                                    data['encoded_classe'].values,
                                                    test_size=0.2,
                                                    random_state=1)

knn = KNeighborsClassifier()
knn.fit(X=X_train.reshape(-1, 1), y=y_train.reshape(-1, 1))

predictions = knn.predict(X=X_test.reshape(-1, 1))

# Get the indexes of the wrong predictions
wrong_indexes = predictions-y_test != 0

# Print predictions accuracy
print(f'Le score de prédiction est: {(y_test.shape[0] - wrong_indexes.sum())/y_test.shape[0]}')

# Get correct prediction to determine which classes are most easily predicted based on sequence length
right_preds = y_test[~wrong_indexes]

# Get the 5 most frequent classes that are rightfully predicted
print(pd.Series(right_preds).value_counts())
fclasses = lencoder.inverse_transform(pd.Series(right_preds).value_counts()[0:5].index)
md(f'Selon ce qu\'on constate, les classe les mieux prédites avec le'+ r' $\textit{KNN}$ basé sur la longueur de la séquence '
   f'sont {fclasses}')

# lets drop the five most easily predicted classes
for cls in fclasses:
    data.drop(index=data[data['classe'] == cls].index, axis=0, inplace=True)

Le score de prédiction est: 0.717948717948718
1     4
3     3
5     3
6     3
7     3
10    3
2     2
4     2
11    2
0     1
8     1
9     1
dtype: int64


/home/ggibeau/.local/lib/python3.8/site-packages/sklearn/neighbors/_classification.py:179: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  return self._fit(X, y)


## b) k-mers

Dans un premier temps, il nous faut trouver toutes les permutations, incluant les répétitions, des nucléotides de l'ARN:

In [8]:
nucleotides = ['A', 'C', 'G', 'U']
max_kmer_length = 4

kmer_lists = list()
# for each length of kmer...
for i in range(0, max_kmer_length):
    # Create all permutations of that length, and the list they will be stored in
    kmer = product(nucleotides, repeat=i+1)
    kmer_lists.append(list())
    l = kmer_lists[i]
    for k in kmer:
        # then store permutated values as string in a list
        w = ''
        for j in range(0, i+1):
            w += k[j]
        l.append(w)

for ix, l in enumerate(kmer_lists):
    print(f'list of {ix+1}-mer')
    for p in l:
        print(p)


list of 1-mer
A
C
G
U
list of 2-mer
AA
AC
AG
AU
CA
CC
CG
CU
GA
GC
GG
GU
UA
UC
UG
UU
list of 3-mer
AAA
AAC
AAG
AAU
ACA
ACC
ACG
ACU
AGA
AGC
AGG
AGU
AUA
AUC
AUG
AUU
CAA
CAC
CAG
CAU
CCA
CCC
CCG
CCU
CGA
CGC
CGG
CGU
CUA
CUC
CUG
CUU
GAA
GAC
GAG
GAU
GCA
GCC
GCG
GCU
GGA
GGC
GGG
GGU
GUA
GUC
GUG
GUU
UAA
UAC
UAG
UAU
UCA
UCC
UCG
UCU
UGA
UGC
UGG
UGU
UUA
UUC
UUG
UUU
list of 4-mer
AAAA
AAAC
AAAG
AAAU
AACA
AACC
AACG
AACU
AAGA
AAGC
AAGG
AAGU
AAUA
AAUC
AAUG
AAUU
ACAA
ACAC
ACAG
ACAU
ACCA
ACCC
ACCG
ACCU
ACGA
ACGC
ACGG
ACGU
ACUA
ACUC
ACUG
ACUU
AGAA
AGAC
AGAG
AGAU
AGCA
AGCC
AGCG
AGCU
AGGA
AGGC
AGGG
AGGU
AGUA
AGUC
AGUG
AGUU
AUAA
AUAC
AUAG
AUAU
AUCA
AUCC
AUCG
AUCU
AUGA
AUGC
AUGG
AUGU
AUUA
AUUC
AUUG
AUUU
CAAA
CAAC
CAAG
CAAU
CACA
CACC
CACG
CACU
CAGA
CAGC
CAGG
CAGU
CAUA
CAUC
CAUG
CAUU
CCAA
CCAC
CCAG
CCAU
CCCA
CCCC
CCCG
CCCU
CCGA
CCGC
CCGG
CCGU
CCUA
CCUC
CCUG
CCUU
CGAA
CGAC
CGAG
CGAU
CGCA
CGCC
CGCG
CGCU
CGGA
CGGC
CGGG
CGGU
CGUA
CGUC
CGUG
CGUU
CUAA
CUAC
CUAG
CUAU
CUCA
CUCC
CUCG
CUCU
CUGA
CUGC
CUGG
CUGU
CUUA
CUUC
CU

Nous allons ensuite créer 4 nouveaux $\textit{dataframes}$ pour faire le décompte du nombre d'occurence des k-mer dans
les séquence de nucléotides, et ultimement pouvoir les 4 présentations.

In [9]:
# Lets start by copying our main dataframe in 4 new dataframes, drop the columns we won't need and put them in a list
df_list = list()
for i in range(0, max_kmer_length):
    df_list.append(data.copy())
    df_list[i].drop(labels=['longueur','encoded_classe'], inplace=True, axis=1)
    # then lets add the needed columns in each new dataframe
    for km in kmer_lists[i]:
        df_list[i][km] = pd.Series(range(0, df_list[0].shape[0]))

In [11]:
# Count the occurence of every k-mer for every sequence
for df, kmlist in zip(df_list, kmer_lists):
    for km in kmlist:
        count = 0
        for ix, row in df.iterrows():
            # Here we use re.finditer() fct to get all k-mer occurence in a sequence
            count = sum(1 for _ in finditer(km, row['sequence']))
            df.loc[ix, km] = count
        # print(df.loc[:, km])

                         A
id                        
AF093014.1/662-809    51.0
M14649.1/2015-2169    42.0
X53361.2/1206-1368    33.0
M86760.1/218-371      39.0
X00601.1/3997-4154    47.0
...                    ...
'U10566.1/72-260"     60.0
'U10567.1/1-190"      55.0
'U22351.1/82-237"     52.0
'U22350.1/54-211"     52.0
AF399707.1/2183-2342  54.0

[112 rows x 1 columns]
